## Libraries

In [1]:
import numpy as np
import pandas as pd
import xlrd
import os

In [2]:
import sys
np.set_printoptions(threshold=np.inf)

## Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#os.listdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
os.listdir(".")

In [ ]:
housePath = "/content/drive/My Drive/Colab Notebooks/house_prices.xlsx"

# Load excel data for House Prices


In [ ]:
# #wb = xlrd.open_workbook(housePath)
# #data = pd.read_excel(housePath)
# xl = pd.ExcelFile(housePath)


### Extract sheets, concat sheets, and pickle them

In [ ]:
# df00 = xl.parse("2000")
# df01 = xl.parse("2001")
# df02 = xl.parse("2002")
# df03 = xl.parse("2003")
# df04 = xl.parse("2004")
# df05 = xl.parse("2005")
# df06 = xl.parse("2006")
# df07 = xl.parse("2007")
# df08 = xl.parse("2008")
# df09 = xl.parse("2009")
# df10 = xl.parse("2010")
# df11 = xl.parse("2011")
# df12 = xl.parse("2012")
# df13 = xl.parse("2013")
# df14 = xl.parse("2014")
# df15 = xl.parse("2015")
# df16 = xl.parse("2016")
# df17 = xl.parse("2017")
# df18 = xl.parse("2018")
# df19 = xl.parse("2019")

In [ ]:
# df00.to_pickle("df00.pkl")
# df01.to_pickle("df01.pkl")
# df02.to_pickle("df02.pkl")
# df03.to_pickle("df03.pkl")
# df04.to_pickle("df04.pkl")
# df05.to_pickle("df05.pkl")
# df06.to_pickle("df06.pkl")
# df07.to_pickle("df07.pkl")
# df08.to_pickle("df08.pkl")
# df09.to_pickle("df09.pkl")
# df10.to_pickle("df10.pkl")
# df11.to_pickle("df11.pkl")
# df12.to_pickle("df12.pkl")
# df13.to_pickle("df13.pkl")
# df14.to_pickle("df14.pkl")
# df15.to_pickle("df15.pkl")
# df16.to_pickle("df16.pkl")
# df17.to_pickle("df17.pkl")
# df18.to_pickle("df18.pkl")
# df19.to_pickle("df19.pkl")

In [ ]:
df00 = pd.read_pickle("./Pickle/df00.pkl")
df01 = pd.read_pickle("./Pickle/df01.pkl")
df02 = pd.read_pickle("./Pickle/df02.pkl")
df03 = pd.read_pickle("./Pickle/df03.pkl")
df04 = pd.read_pickle("./Pickle/df04.pkl")
df05 = pd.read_pickle("./Pickle/df05.pkl")
df06 = pd.read_pickle("./Pickle/df06.pkl")
df07 = pd.read_pickle("./Pickle/df07.pkl")
df08 = pd.read_pickle("./Pickle/df08.pkl")
df09 = pd.read_pickle("./Pickle/df09.pkl")
df10 = pd.read_pickle("./Pickle/df10.pkl")
df11 = pd.read_pickle("./Pickle/df11.pkl")
df12 = pd.read_pickle("./Pickle/df12.pkl")
df13 = pd.read_pickle("./Pickle/df13.pkl")
df14 = pd.read_pickle("./Pickle/df14.pkl")
df15 = pd.read_pickle("./Pickle/df15.pkl")
df16 = pd.read_pickle("./Pickle/df16.pkl")
df17 = pd.read_pickle("./Pickle/df17.pkl")
df18 = pd.read_pickle("./Pickle/df18.pkl")
df19 = pd.read_pickle("./Pickle/df19.pkl")

In [ ]:
dfList = [df00,df01,df02,df03,df04,df05,df06,
         df07,df08,df09,df10,df11,df12,df13,
         df14,df15,df16,df17,df18,df19]

rawHouse = pd.concat(dfList)
rawHouse.to_pickle('rawHouse.pkl')

In [ ]:
rawHouse.info()

## Clean house from pickle

In [ ]:
house = pd.read_pickle('rawHouse.pkl')

colList = ['PARCEL','GARAGE 1 TYPE','GARAGE 2 TYPE']

house['Age'] = house['YEAR'] - house['YR BUILT']

house['YEAR'] = house['YEAR'].astype(str)
house['YEAR'] = house['YEAR'].astype('datetime64[Y]')


for col in colList:
    house[col] = house[col].astype('category')

house.iloc[:,4:14] = house.iloc[:,4:14].astype('category')

# Remove houses that have TAV = $0 and only single family dwellings
house = house.loc[house['TAV'] != 0]
house = house.loc[house['DWELLINGS'].str.contains('SFD')]

house = house.drop(['SUFFIX', 'DWELLINGS'], axis = 1)


house.to_pickle('house.pkl')
#house.head(5)
house.info()

In [ ]:
#print(house.dtypes)
#house.head(5)
#house.iloc[15:20,:]
#house.SUFFIX.unique()


In [ ]:
house.DWELLINGS.unique()

## Build Neighborhoods

Code for 1st source of neighborhood data. Being replaced with Group_List.xlsx

In [ ]:
 xlG = pd.ExcelFile('./Data/GROUP_LIST.xlsx')
groupList = xlG.parse('GROUP')

groupList.to_pickle('groupList.pkl')
groupList.head()

### Join groupList.pkl to house.pkl, to make houseAll.pkl

In [ ]:
groupList = pd.read_pickle('groupList.pkl')
house = pd.read_pickle('house.pkl')

houseAll = pd.merge(house, groupList, on = 'GROUP', how = 'left')

colList = ['GROUP', 'GEO-ECONOMIC AREA', 'MLS AREA']

for col in colList:
    houseAll[col] = houseAll[col].astype('category')
    
houseAll = houseAll[houseAll['YEAR'] != '2019-01-01']    

houseAll = houseAll.rename(columns={'GEO-ECONOMIC AREA' : 'Neighborhood','MLS AREA' : 'Area',
                                    'YEAR' : 'Year', 'TOTAL SQ FT' : 'Total_Sq_Ft',
                                    'YR BUILT' : 'Year_Built', 'BEDROOMS' : 'Bedroom_Count',
                                   'PARCEL' : 'Parcel', 'GROUP' : 'Group', 'BATHROOMS' : 'Bathroom_Count'})

houseAll.to_pickle('houseAll.pkl')
houseAll.head(5)

In [ ]:
houseAll.tail()

# Find % change in TAV from prev year

In [10]:
houseAll = pd.read_pickle('houseAll.pkl')

df = houseAll[['Parcel','Year','Group', 'Neighborhood','Area', 'Age','Bedroom_Count','Bathroom_Count','TAV']]

df['TAV_Ch'] = df.groupby('Parcel')['TAV'].pct_change()

df.to_pickle('abvHouse.pkl')
df.head(5)

C:\Users\delightb\AppData\Local\Continuum\anaconda3\envs\tensorenviron\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Parcel,Year,Group,Neighborhood,Area,Age,Bedroom_Count,Bathroom_Count,TAV,TAV_Ch
0,R0017650020,2000-01-01,AARON PARK SUB,NORTH CENTRAL BENCH,W BOISE,2,4,4.00,158000.0,NaN
1,R0017650030,2000-01-01,AARON PARK SUB,NORTH CENTRAL BENCH,W BOISE,1,4,4.00,165800.0,NaN
2,R0017650040,2000-01-01,AARON PARK SUB,NORTH CENTRAL BENCH,W BOISE,1,4,4.00,165800.0,NaN
3,R0017650050,2000-01-01,AARON PARK SUB,NORTH CENTRAL BENCH,W BOISE,2,4,4.00,155000.0,NaN
4,R0027000008,2000-01-01,ABBS SUB,CENTRAL BOISE BENCH,BOISE BENCH,63,1,1.25,129100.0,NaN


In [12]:
parcelNum = 'R2024320455'
x = pd.read_pickle('abvHouse.pkl')
x = x[x['Parcel'] == 'R0027000008']
x = x[['Parcel','Year','TAV','TAV_Ch']]
x

,Parcel,Year,TAV,TAV_Ch
4,R0027000008,2000-01-01,129100.0,NaN
54677,R0027000008,2001-01-01,145600.0,0.127808
110200,R0027000008,2002-01-01,146700.0,0.007555
166351,R0027000008,2003-01-01,161700.0,0.102249
223313,R0027000008,2004-01-01,165200.0,0.021645
281777,R0027000008,2005-01-01,175500.0,0.062349
343520,R0027000008,2006-01-01,225800.0,0.286610
405877,R0027000008,2007-01-01,237600.0,0.052259
468969,R0027000008,2008-01-01,237600.0,0.000000
532710,R0027000008,2009-01-01,175900.0,-0.259680
